In [16]:
# Bigdata healthcare, Descriptive Sattistics
import time
import pandas as pd
import numpy as np
import sys
print(sys.version)
#import os
#os.getcwd()

3.7.7 (default, May  6 2020, 11:45:54) [MSC v.1916 64 bit (AMD64)]


In [17]:


def read_csv(filepath):
    '''
    Read the events.csv and mortality_events.csv files. 
    Variables returned from this function are passed as input to the metric functions.
    '''
    events = pd.read_csv(filepath + 'events.csv')
    mortality = pd.read_csv(filepath + 'mortality_events.csv')
    return events, mortality

def event_count_metrics(events, mortality):
    '''
    Implement this function to return the event count metrics.
    Event count is defined as the number of events recorded for a given patient.
    '''
    events_2 = events.copy()
    events_2 = events_2.assign(deceased=events_2['patient_id'].isin(mortality['patient_id']).astype(int))
    surviveData = events_2.copy()
    surviveData = surviveData.loc[surviveData['deceased'] == 0]
    deceasedData = events_2.copy()
    deceasedData = deceasedData.loc[deceasedData['deceased'] == 1]

    deceasedEventCount = deceasedData['event_id'].count()
    deceasedDistincPatient = deceasedData['patient_id'].nunique()
    deceasedData['Agg'] = deceasedData.groupby('patient_id')['patient_id'].transform('count')
    
    Meandeceased_EventC = deceasedEventCount/deceasedDistincPatient
    Maxdeceased_EventC = deceasedData['Agg'].max()
    Mindeceased_EventC = deceasedData['Agg'].min()

    surviveEventCount = surviveData['event_id'].count()
    surviveDistincPatient = surviveData['patient_id'].nunique()
    surviveData['Agg'] = surviveData.groupby('patient_id')['patient_id'].transform('count')

    Meansurvive_EventC = surviveEventCount/surviveDistincPatient
    Maxsurvive_EventC = surviveData['Agg'].max()
    Minsurvive_EventC = surviveData['Agg'].min()

    return Mindeceased_EventC, Maxdeceased_EventC, Meandeceased_EventC, Minsurvive_EventC, Maxsurvive_EventC, Meansurvive_EventC

def encounter_count_metrics(events, mortality):
    '''
    Implement this function to return the encounter count metrics.
    Encounter count is defined as the count of unique dates on which a given patient visited the ICU. 
    '''
    events_2 = events.copy()
    events_2 = events_2.assign(deceased=events_2['patient_id'].isin(mortality['patient_id']).astype(int))
    surviveData = events_2.copy()
    surviveData = surviveData.loc[surviveData['deceased'] == 0]
    surviveData = surviveData.drop(['deceased', 'event_id', 'event_description', 'value'], axis=1)
    surviveData = surviveData.drop_duplicates()
    deceasedData = events_2.copy()
    deceasedData = deceasedData.loc[deceasedData['deceased'] == 1]
    deceasedData = deceasedData.drop(['deceased', 'event_id', 'event_description', 'value'], axis=1)
    deceasedData = deceasedData.drop_duplicates()
    
    deceased_timestamp = deceasedData['timestamp'].count()
    deceased_patients = deceasedData['patient_id'].nunique()
    deceasedData['Agg'] = deceasedData.groupby('patient_id')['patient_id'].transform('count')
    
    Meandeceased_EncounC = deceased_timestamp/deceased_patients
    Maxdeceased_EncounC = deceasedData['Agg'].max()
    Mindeceased_EncounC = deceasedData['Agg'].min()

    survive_timestamp = surviveData['timestamp'].count()
    survive_patients = surviveData['patient_id'].nunique()
    surviveData['Agg'] = surviveData.groupby('patient_id')['patient_id'].transform('count')
    
    Meansurvive_EncounC = survive_timestamp/survive_patients
    Maxsurvive_EncounC = surviveData['Agg'].max()
    Minsurvive_EncounC = surviveData['Agg'].min()

    return Mindeceased_EncounC, Maxdeceased_EncounC, Meandeceased_EncounC, Minsurvive_EncounC, Maxsurvive_EncounC, Meansurvive_EncounC

def record_length_metrics(events, mortality):
    '''
    Implement this function to return the record length metrics.
    Record length is the duration between the first event and the last event for a given patient. 
    '''
    events_2 = events.copy()
    events_2['timestamp'] = pd.to_datetime(events_2['timestamp'])    
    LengthMetrics = events_2[['patient_id','timestamp']].groupby('patient_id').agg([max,min])
    LengthMetrics['duration'] = LengthMetrics['timestamp']['max']-LengthMetrics['timestamp']['min']
    LengthMetrics['days']=LengthMetrics['duration'].dt.days
    LengthMetrics=LengthMetrics.reset_index()
    LengthMetrics = LengthMetrics[['patient_id', 'days']]
    deathData1 = LengthMetrics[LengthMetrics['patient_id'].isin(mortality['patient_id'])]    
    deathData = deathData1['days']  
    Meandeceased_LenMetrics = deathData.mean()
    Maxdeceased_LenMetrics = deathData.max()
    Mindeceased_LenMetrics = deathData.min()
    surviveData = LengthMetrics[-LengthMetrics['patient_id'].isin(mortality['patient_id'])]
    surviveData = surviveData['days']
    Meansurvive_LenMetrics = surviveData.mean()
    Maxsurvive_LenMetrics = surviveData.max()
    Minsurvive_LenMetrics = surviveData.min()    
    return Mindeceased_LenMetrics, Maxdeceased_LenMetrics, Meandeceased_LenMetrics, Minsurvive_LenMetrics, Maxsurvive_LenMetrics, Meansurvive_LenMetrics

def main():
    
    train_path = 'data/'
    events, mortality = read_csv(train_path)

    #event count metrics
    print("output for the event count metrics is :") 
    print(event_count_metrics(events, mortality))
    print()
    #encounter count metrics    
    print("output for the encounter count metrics is :")
    print(encounter_count_metrics(events, mortality)) 
    print()
    #record length metrics    
    print("output for the record length metrics is :")
    print( record_length_metrics(events, mortality))
    
    
if __name__ == "__main__":
    main()

output for the event count metrics is :
(1, 8635, 982.014, 1, 12627, 498.118)

output for the encounter count metrics is :
(1, 203, 23.038, 1, 391, 15.452)

output for the record length metrics is :
(0, 1972, 127.532, 0, 2914, 159.2)
